In [1]:
import pandas as pd

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import pickle
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, LSTM, Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
# 한글폰트 사용 in colab
%matplotlib inline  

import matplotlib as mpl 
import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  

!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()
mpl.rcParams['axes.unicode_minus'] = False

Selecting previously unselected package fonts-nanum.
(Reading database ... 155629 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...
NanumBarunGothic


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
상승_공격투자형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/상승/공격투자형.csv')
상승_극안전형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/상승/극안전형.csv')
상승_안전추구형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/상승/안전추구형.csv')
상승_위험중립형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/상승/위험중립형.csv')
상승_적극투자형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/상승/적극투자형.csv')

In [7]:
최근하락_공격투자형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/최근하락/공격투자형.csv')
최근하락_극안전형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/최근하락/극안전형.csv')
최근하락_안전추구형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/최근하락/안전추구형.csv')
최근하락_위험중립형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/최근하락/위험중립형.csv')
최근하락_적극투자형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/최근하락/적극투자형.csv')

In [8]:
하락_공격투자형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/하락/공격투자형.csv')
하락_극안전형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/하락/극안전형.csv')
하락_안전추구형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/하락/안전추구형.csv')
하락_위험중립형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/하락/위험중립형.csv')
하락_적극투자형 = pd.read_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/하락/적극투자형.csv')

In [18]:
공격투자형 = pd.concat([하락_공격투자형,상승_공격투자형,최근하락_공격투자형],axis=0)
극안전형 = pd.concat([하락_극안전형,상승_극안전형,최근하락_극안전형],axis=0)
안전추구형 = pd.concat([하락_안전추구형,상승_안전추구형,최근하락_안전추구형],axis=0)
위험중립형 = pd.concat([하락_위험중립형,상승_위험중립형,최근하락_위험중립형],axis=0)
적극투자형 = pd.concat([하락_적극투자형,상승_적극투자형,최근하락_적극투자형],axis=0)

In [24]:
공격투자형.to_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/투자유형비중총결합/공격투자형.csv')
극안전형.to_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/투자유형비중총결합/극안전형.csv')
안전추구형.to_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/투자유형비중총결합/안전추구형.csv')
위험중립형.to_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/투자유형비중총결합/위험중립형.csv')
적극투자형.to_csv('/content/gdrive/MyDrive/FINAL TEAM/파이널프로젝트/비주식팀/Data/비중DF/투자유형비중총결합/적극투자형.csv')